In [1]:
import pandas as pd
# import scprep as sc
import scanpy as sc
import phate
import numpy as np
import seaborn as sns
import scprep
from sklearn.metrics import roc_auc_score, average_precision_score
import os
from tqdm import tqdm
import warnings


In [2]:
from tqdm import tqdm
import warnings
from statsmodels.tsa.stattools import grangercausalitytests

lag_order = 1 # since we aggregated the data in to 9 bins we only need 1 lag
maxlag = (
    lag_order,  # becuase we got this value before. We are not suppose to add 1 to it
)
test = "ssr_chi2test"

from joblib import Parallel, delayed

def grangers_causation_matrix(
    data, in_variables, out_variables, test="ssr_chi2test", n_jobs=1, warn=False
):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """

    def get_pval(dd):
        if warn:
            test_result = grangercausalitytests(dd, maxlag=maxlag, verbose=True)
        else:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=FutureWarning)
                test_result = grangercausalitytests(dd, maxlag=maxlag, verbose=False)
                # according to the documentation https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.grangercausalitytests.html,
                # the dd has 2 columns, second causes the first.
                # print(test_result.keys())
                # print(f'dd shape {dd.shape}')
                # print(test_result[1][1][0].params[0])

                # print(test_result[1][1][0].summary())
                # print(test_result[1][1][1].summary())
                # print(test_result[1][1][1].model.exog_names)

                # print(test_result[2][1][1].summary())
                


                # print(test_result[1][1][0].model.params)
                # print(test_result[1])
                # print(test_result[1][1][0].params)
                # print(test_result[1][1][0].params['constant'])
                # print(test_result[1][1][0].model.exog_names)
        # import pdb; pdb.set_trace()
        # print(dd.columns)
        # print(dd.head())
        # print(test_result[1][1][1].params)
        p_values = [test_result[i][0][test][1] for i in maxlag] # test_result[i][1] is the unrestricted model, test_result[i][1][0] is the restricted model
        # coefs = [test_result[i][1][1].params[0] for i in maxlag]
        coefs = [test_result[i][1][1].params[1] for i in maxlag] # x1, x2, const

        arg_min_p_value = np.argmin(p_values)
        min_p_value = p_values[arg_min_p_value]
        min_coef = coefs[arg_min_p_value]
        # print(p_values)
        # print(coefs)
        return (min_p_value, min_coef)


    out = Parallel(n_jobs=n_jobs)(
        # delayed(get_pval)(data[[r, c]]) # this is incorrect
        delayed(get_pval)(data[[c, r]]) # this means r causes c, so r is be in and c is out
        for c in tqdm(out_variables, desc="Processing columns")  # Outer loop progress bar
        for r in tqdm(in_variables, desc="Processing rows", leave=False)  # Inner loop progress bar
    )
    # Note that this is the wrong way and must be corrected
    # df = pd.DataFrame(
    #     np.array(out).reshape((len(in_variables), len(out_variables))), # this is incorrect
    #     columns=out_variables,
    #     index=in_variables,
    # )
    out_p = [p for (p,c) in out]
    out_c = [c for (p,c) in out]
    df_p = pd.DataFrame(
        np.array(out_p).reshape((len(out_variables), len(in_variables))), # should be reshaped to len(out_variables), len(in_variables) according to the for loop.
        columns=in_variables,
        index=out_variables,
    ).T # used the correct reshaping, and then transposed the matrix so the x and y are semantically correct (x causes y).
    df_c = pd.DataFrame(
        np.array(out_c).reshape((len(out_variables), len(in_variables))), # should be reshaped to len(out_variables), len(in_variables) according to the for loop.
        columns=in_variables,
        index=out_variables,
    ).T
    df_p.index = [var + "_x" for var in in_variables]
    df_p.columns = [var + "_y" for var in out_variables]
    df_c.index = [var + "_x" for var in in_variables]
    df_c.columns = [var + "_y" for var in out_variables]
    return df_p, df_c

def do_granger(trajs, in_genes, out_genes, n_jobs=1, warn=False):
    # in causes out
    trajs = trajs.T[::10]
    trajs = trajs - trajs.shift(1)
    trajs = trajs.dropna()
    out_traj_p, out_traj_c = grangers_causation_matrix(
        trajs, in_variables=in_genes, out_variables=out_genes, n_jobs=n_jobs, warn=warn
    )
    return out_traj_p, out_traj_c



In [3]:
# traj_gene_sp = np.load("../data_human_samples/10-trajectories_gene_space.npy")
traj_gene_sp = np.load(f"../results_final_Dec22/traj_gene_space_T_extreme.npy", allow_pickle=True)

In [4]:
adata = sc.read_h5ad('../data/rna_figure_ready.h5ad')

In [5]:
# sc.pp.highly_variable_genes(adata, n_top_genes=400)
alex_genes = open('../data/alex_genes.txt').read().splitlines()
alex_tfs = open('../data/alex_tfs.txt').read().splitlines()

In [6]:
# gene_mask = np.isin(adata.var_names, alex_genes)
# tf_mask = np.isin(adata.var_names, alex_tfs)

In [7]:
alex_gene_all = list(set(alex_genes) | set(alex_tfs))
alex_gene_all_mask = np.isin(adata.var_names, alex_gene_all)
traj_subset = traj_gene_sp[:, :, alex_gene_all_mask]
gene_names = adata.var_names[alex_gene_all_mask]

In [8]:
traj_subset.shape

(100, 19, 2226)

In [9]:
# traj_df = pd.DataFrame(np.transpose(traj_subset, (1, 0, 2)).mean(axis=0), columns=gene_names)
data = np.transpose(traj_subset, (1, 0, 2))
valid_mask = (data.mean(axis=0).var(axis=0) != 0.0)
data = data[:, :, valid_mask]
gene_names = np.array(gene_names)[valid_mask]
traj_df = pd.DataFrame(data.mean(axis=0), columns=gene_names)

In [10]:
gm_p, gm_c = do_granger(traj_df.T, in_genes=alex_tfs, out_genes=alex_genes, n_jobs=-1)
gm_p.to_csv(f'../results_final_Dec22/granger_T_extreme_p.csv')
gm_c.to_csv(f'../results_final_Dec22/granger_T_extreme_c.csv')

Processing columns:   0%|          | 0/1970 [00:00<?, ?it/s]

Processing rows:   0%|          | 0/331 [00:00<?, ?it/s]

Processing rows:   2%|▏         | 8/331 [00:00<00:05, 59.12it/s]

Processing rows:   5%|▍         | 16/331 [00:02<00:48,  6.44it/s]

Processing rows:  12%|█▏        | 40/331 [00:02<00:13, 21.12it/s]

...

Processing columns: 100%|█████████▉| 1969/1970 [06:56<00:00,  5.13it/s]

Processing rows:   0%|          | 0/331 [00:00<?, ?it/s]

Processing rows:  37%|███▋      | 124/331 [00:00<00:00, 443.82it/s]

Processing columns: 100%|██████████| 1970/1970 [06:57<00:00,  3.98it/s]

Processing columns: 100%|██████████| 1970/1970 [06:57<00:00,  4.72it/s]